In [ ]:
#sla de locations op
import pandas as pd
# from geopy.geocoders import Nominatim
import requests
import pandas as pd
import time
import json
# geolocator = Nominatim(user_agent="MyApp")

start_time = '20180101'
end_time = '20181231'
# start_time = '20190101'
# end_time = '20191231'
# start_time = '20200101'
# end_time = '20201231'
# start_time = '20210101'
# end_time = '20210731'
# start_time = '20220101'
# end_time = '20220731'
# start_time = '20230101'
# end_time = '20230731'





df_generation_plants = pd.read_excel("C:\\Users\\LIAT-ER\\Documents\\Thesis_Thibo\\weather\\Listado_Recursos_Generacion.xlsx", skiprows=3)
locations = list(df_generation_plants['Municipio'].unique())

with open('json_locations_complete.json', 'r') as fp:
    json_locations = json.load(fp)


with open('json_locations_complete.json', 'r') as fp:
    json_locations = json.load(fp)
for city in locations:

    ldf = pd.DataFrame()
    lon = json_locations[city]['longitude']
    lat = json_locations[city]['latitude']


import pandas as pd
import requests
import json
from concurrent.futures import ThreadPoolExecutor

result_data = []

max_workers = 32  # You can adjust the number of worker threads based on your system's capabilities
cities_with_errors = []
def fetch_city_data(city_info):
    city, city_info = city_info
    print(city)
    print("-------  ",+ locations.index(city)/(len(locations))*100,"%  -------")
    lon = city_info['longitude']
    lat = city_info['latitude']
    url = "https://power.larc.nasa.gov/api/temporal/hourly/point?parameters=QV2M,RH2M,PRECTOTCORR,ALLSKY_SFC_PAR_TOT,ALLSKY_SFC_SW_DWN,ALLSKY_SRF_ALB,T2M,T2MDEW,T2MWET,WS10M,WD10M,WS50M&community=SB&longitude={}&latitude={}&start={}&end={}&format=JSON".format(lon, lat, start_time, end_time)
    response = requests.get(url)
    data = response.json()
    if city == 'FLORIDA':
        print(data)
    else:
        pass
    city_result_data = []
    try:
        for param, param_data in data['properties']['parameter'].items():
            param_values = list(param_data.values())
            param_times = list(param_data.keys())
            city_result_data.extend(
                [{'Datetime': pd.to_datetime(t, format='%Y%m%d%H'), 'City': city, param: value}
                 for t, value in zip(param_times, param_values)]
            )
    except Exception as e:
        print(e)
        print(city)
        cities_with_errors.append(city)
    
    return city_result_data

with open('json_locations_complete.json', 'r') as fp:
    json_locations = json.load(fp)

city_info_list = list(json_locations.items())

with ThreadPoolExecutor(max_workers=max_workers) as executor:
    city_data_lists = executor.map(fetch_city_data, city_info_list)

for city_data_list in city_data_lists:
    result_data.extend(city_data_list)

result_df = pd.DataFrame(result_data)
result_df = result_df.groupby(['Datetime', 'City']).agg({
    'QV2M': 'first',           
    'RH2M': 'first',
    'PRECTOTCORR': 'first',
    'ALLSKY_SFC_PAR_TOT': 'first',
    'ALLSKY_SFC_SW_DWN': 'first',
    'ALLSKY_SRF_ALB': 'first',
    'T2M': 'first',
    'T2MDEW': 'first',
    'T2MWET': 'first',
    'WS10M': 'first',
    'WD10M': 'first',
    'WS50M': 'first'
}).reset_index()



result_df.to_csv('result_all_locations_'+start_time+'_'+end_time+'.csv')